<h2> <center>Understanding use of cython library and a test case of its performance</center></h2>


- The actual code for the project is from Siraj Raval please view this https://github.com/llSourcell/c_programming_for_machine_learning/blob/master/Cython.ipynb for more details . 
    This is just a basic setup and experimentation

In [1]:
from random import random

<h3>A recatngle example using simple python<?h3>

In [2]:
threshold = 0.3


def calc_area(length, width):
    area = length * width
    return area

def main_rectangles_py():
    n_rectangles = 10000000
    out = 0
    lengths = list(random() for i in range(n_rectangles))
    breadths = list(random() for j in range(n_rectangles))
    for i in range(n_rectangles):
        area = calc_area(lengths[i], breadths[i])
        if area > threshold:
            out += 1
    return out/n_rectangles



In [3]:
%%time
main_rectangles_py()

CPU times: user 3.61 s, sys: 180 ms, total: 3.79 s
Wall time: 3.79 s


0.3388324

<h2>Cython</h2>

<h3>Why is Cython better ?</h3>

- python uses garbage collection instead of manual memory management, which means developers can freely create objectsand Python's memory manager will periodically look for any objects that are no longer referenced by their program this overhead makes demands on the runtime environment (slower).
- explicit C data type

<h3>Basics</h3>

- cython is Python with C data types.
- can be called in a jupyter notebook using %load_ext Cython

In [4]:
%load_ext Cython

- Then, prefix a cell with the %%cython marker to compile it:

In [14]:
%%time
%%cython

cdef float a = 0
for i in range(100000000):
    a += i
print(a)

2251799813685248.0
CPU times: user 5.56 s, sys: 7.95 ms, total: 5.57 s
Wall time: 5.56 s


In [6]:
%%time

a = 0 #exectued using python compiler
for i in range(100000000):
    a += i
print(a)

4999999950000000
CPU times: user 7.95 s, sys: 345 µs, total: 7.95 s
Wall time: 7.95 s


<p><b>As we can see, cython compiler uses 5 s seconds to complete the same operation that the python compiler takes almost twice the time of 8 s</p>

<h3>A rectangle example using Cython</h3>

In [7]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


- <b>cdef</b> - cython only functions, can't access these from python-only code, must access within Cython, since there will be no C translation to Python for these.
- <b>cymem</b> - cymem provides two small memory-management helpers for Cython. They make it easy to tie memory to a Python object's life-cycle, so that the memory is freed when the object is garbage collected
- <b>pool</b> - The Pool object saves the memory addresses internally, and frees them when the object is garbage collected.  This is particularly handy for deeply nested structs, which have complicated initialization functions. Just pass the Pool object into the initializer, and you don't have to worry about freeing your struct at all — all of the calls to Pool.alloc will be automatically freed when the Pool expires.


<h4> Using pool for mem allocation</h4>

In [8]:
# from cymem.cymem cimport Pool
# cdef Pool mem = Pool()
# data1 = <int*>mem.alloc(10, sizeof(int))
# data2 = <float*>mem.alloc(12, sizeof(float))

In [9]:
%%cython
from cymem.cymem cimport Pool


In [10]:
%%cython
from cymem.cymem cimport Pool #memory management helper
from random import random

# create a rectangle constructor
cdef struct Rectangle:
    #explicitly declare c datatypes 
    float height
    float width

# definition for checking rectangles
cdef int check_rectangles_cy(Rectangle* rectangles, int n_rectangles, float threshold):
    out = 0
    for rectangle in rectangles[: n_rectangles]:
        if rectangle.width * rectangle.height > threshold:
            out += 1
    return out

#main program (no cdef)
def main_rectangles_cy():
    cdef int n_rectangles = 10000000
    cdef float threshold = 0.3
    # The Pool Object will save memory addresses internally
    cdef Pool mem = Pool()
    cdef Rectangle*  rectangles = <Rectangle*>mem.alloc(n_rectangles, sizeof(Rectangle))
    for i in range(n_rectangles):
        rectangles[i].width = random()
        rectangles[i].height = random()
    n_out = check_rectangles_cy(rectangles, n_rectangles, threshold)
    print(n_out)

In [11]:
%%time
main_rectangles_cy()

3387285
CPU times: user 602 ms, sys: 36 ms, total: 638 ms
Wall time: 637 ms


<b>Our cython implimentation takes 638 seconds. Now lets compare the python implementation

In [12]:
%%time
main_rectangles_py()

CPU times: user 3.46 s, sys: 184 ms, total: 3.64 s
Wall time: 3.64 s


0.339123

<b>Whereas, python implementation takes 3640 s to run a similar program => a speed up of almost 6x</b>